In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else ''
print(device)
block_size=8
batch_size=4

cuda


In [8]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
 
def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)                                                                                                                                           

tensor([ 66727, 136540, 127727, 124959])
inputs:
tensor([[73, 73, 65, 58,  1, 66, 54, 67],
        [67, 57, 10, 10, 76, 61, 54, 73],
        [72,  1, 61, 58, 54, 57,  1, 76],
        [62, 67, 60,  1, 76, 58,  1, 66]], device='cuda:0')
targets:
tensor([[73, 65, 58,  1, 66, 54, 67,  9],
        [57, 10, 10, 76, 61, 54, 73,  1],
        [ 1, 61, 58, 54, 57,  1, 76, 54],
        [67, 60,  1, 76, 58,  1, 66, 62]], device='cuda:0')


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,index,targets):
        logits=self.token_embedding_table(index)
        B,T,C=logits.shape
        logits=logits.view(B*T,C)
        targets=targets.view(B*T)
        loss=F.cross_entropy(logits,targets)
        return logits

when input is  tensor([80]) target is  tensor(1)
when input is  tensor([80,  1]) target is  tensor(1)
when input is  tensor([80,  1,  1]) target is  tensor(28)
when input is  tensor([80,  1,  1, 28]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)
